In [93]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed

Read in restaurant reviews data_frame from reviews_NLP_tagging.ipynb and only take 5 star reviews (highly recommended)

In [75]:
rest_reviews = pd.read_pickle('reviews_for_freq_user_merge.p')

In [76]:
fivestar_reviews = rest_reviews[rest_reviews['stars_x']== 5.0]

Read in coefficients for each user analyzed

In [8]:
freq_users_coef = pd.read_pickle('freq_users_coef.p')

Function to grab Taste Doppelgangers sorts dictionary by correlation between users'coeficient and other users'

Reutrns sorted dictionary where key is user id and value is correlation

In [106]:
def get_doppelgangers(User_ID, freq_users_coef):
    User_Coef = freq_users_coef[User_ID]
    Dopplegangers_Dict = {k: np.corrcoef(v, User_Coef)[0,1] for k, v in \
                          sorted(freq_users_coef.items(), key = lambda kv: -np.corrcoef(kv[1], User_Coef)[0,1])}
    return Dopplegangers_Dict

Goes through user ids in Dopplegangers_Dict grabs doppelganger and searches their reviews for restaurant that matches requirements

Returns matching restaurant name and doppleganger's user id

In [108]:
def get_recommendation(User_ID, Dopplegangers_Dict, Restaurant_Type, fivestar_reviews):
    for user_id in Dopplegangers_Dict.keys():
        if user_id != User_ID:
            doppleganger_recs = fivestar_reviews[fivestar_reviews['user_id'] == user_id]
            matches = doppleganger_recs[doppleganger_recs['categories'].str.contains(Restaurant_Type, na=False)]\
                .sort_values(by = 'stars_y', ascending=False)['name'].to_list()
            if matches:
                return matches[0], user_id

Puts Everything together
Returns name of recommended restaurant and score of dopplganger that gave it (correlation*100)

In [124]:
def get_doppel_rec(User_ID,Restaurant_Type, freq_users_coef, fivestar_reviews):
    Dopplegangers_Dict = get_doppelgangers(User_ID, freq_users_coef)
    rest_rec, dopple_id = get_recommendation(User_ID, Dopplegangers_Dict, Restaurant_Type, fivestar_reviews)
    return f'Your Taste Doppleganger highly recommends: {rest_rec} with a doppleganger score of {np.int(Dopplegangers_Dict[dopple_id]*100)}%'

Toy example: Change User ID requesting recomendation and Restaurant Type and see how recommendation changes

In [125]:
interact(get_doppel_rec, User_ID=freq_users,\
         Restaurant_Type = ['Japanese', 'Chinese', 'American', 'Italian', 'Pizza', 'Indian', 'Coffee', 'Vietnamese','Mexican'],\
         freq_users_coef = fixed(freq_users_coef),\
         fivestar_reviews = fixed(fivestar_reviews))

interactive(children=(Dropdown(description='User_ID', options=('CxDOIDnH8gp9KXzpBHJYXw', 'bLbSNkLggFnqwNNzzq-I…

<function __main__.get_doppel_rec(User_ID, Restaurant_Type, freq_users_coef, fivestar_reviews)>